In [1]:
!pip install git+https://github.com/huggingface/transformers.git@main accelerate

  Cloning https://github.com/huggingface/transformers.git (to revision main) to /tmp/pip-req-build-6xzyd2cy
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-6xzyd2cy
  Resolved https://github.com/huggingface/transformers.git to commit 96eb06286b63c9c93334d507e632c175d6ba8b28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)


In [ ]:

from transformers import AutoTokenizer
import transformers
import torch

In [ ]:

model = "codellama/CodeLlama-13b-Instruct-hf"

In [ ]:

tokenizer = AutoTokenizer.from_pretrained(model,
                                          padding_side = "left")

pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.59k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/9.90G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/6.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [2]:

system_prompt = """
Provide answers in Cypher based on the following schema and example.

### The Schema

Nodes and their properties are the following:

{{'properties': ['id', 'name'], 'labels': 'Journal'}}
{{'properties1': ['id', 'name'], 'labels': 'Author'}}
{{'properties': ['id', 'title', 'abstract', 'authors', 'date', 'journal'], 'labels': 'Article'}}

The relationships are the following:

{{'source': 'Journal', 'relationship': 'PUBLISHED', 'target': ['Article']}}
{{'source': 'Author', 'relationship': 'WROTE', 'target': ['Article']}}

### Example

Question: Fetch 5 journal titles.
Cypher Query: MATCH (j:Journal) RETURN j.name LIMIT 5
"""

prompt_format = "[INST]<>\n{system}\n<>\n\n{user}[/INST]\n\n".format(system=system_prompt,
                                                                        user='{user}')

In [3]:
prompt_format

"[INST]<>\n\nProvide answers in Cypher based on the following schema and example.\n\n### The Schema\n\nNodes and their properties are the following:\n\n{{'properties': ['id', 'name'], 'labels': 'Journal'}}\n{{'properties1': ['id', 'name'], 'labels': 'Author'}}\n{{'properties': ['id', 'title', 'abstract', 'authors', 'date', 'journal'], 'labels': 'Article'}}\n\nThe relationships are the following:\n\n{{'source': 'Journal', 'relationship': 'PUBLISHED', 'target': ['Article']}}\n{{'source': 'Author', 'relationship': 'WROTE', 'target': ['Article']}}\n\n### Example\n\nQuestion: Fetch 5 journal titles.\nCypher Query: MATCH (j:Journal) RETURN j.name LIMIT 5\n\n<>\n\n{user}[/INST]\n\n"

In [ ]:

def gen_text(prompt, **kwargs):

    full_prompt = prompt_format.format(user=prompt)

    # The default max length is pretty small, increase the threshold
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 512

    output = pipeline(full_prompt,
                      do_sample=True,
                      top_k=10,
                      top_p=0.95,
                      **kwargs)
    output = output[0]["generated_text"]
    return output.split('<>', 1)[1]


In [ ]:

!pip install neo4j


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.0/203.0 kB 3.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for neo4j: filename=neo4j-5.20.0-py3-none-any.whl size=280771 sha256=2d18920492a7b3350e089467b8e824f85d90edbd3553e769167064d6ddcf7ce1
  Stored in directory: /root/.cache/pip/wheels/cb/12/66/764554d079caad4b9a11a02cfc0d200dd876d12935b9cf7e64
Successfully built neo4j


In [ ]:

import pandas as pd
import numpy as np
import json
from datetime import datetime
import hashlib
import copy


In [ ]:

# Load and mount the drive helper
from google.colab import drive

# This will prompt for authorization
drive.mount('/content/drive')



Mounted at /content/drive


In [ ]:

# Load the file containing the selected publications
filename = "/content/drive/MyDrive/intern_quantumics/arxiv_math_reduced.csv"
df = pd.read_csv(filename)

In [ ]:

# Parse authors

lsa = list(df.authors)
ls_authors = []

for e in lsa:
    sep = ";"
    if ";" in e:
        sep = ";"
    else:
        sep = ","

    ls_authors.extend(e.split(sep))

print(len(ls_authors))

ls_authors = list(set(ls_authors))
print(len(ls_authors))

def hash_text(text):
   return hashlib.sha256(str(text).encode('utf-8')).hexdigest()

authors = []

for e in ls_authors:
    authors.append({"name":e,"id":hash_text(e)})


12746
9600


In [ ]:

# Parse journals

journals = []
for j in list(df.journal.unique()):
   journals.append({"name":j,"id":hash_text(j)})
journals[20]

{'name': 'J. Algebra 231, 67-85 (2000)',
 'id': '23af3d6d450a2a8eb71d4baf5b25057daaada9fa050f130f5d4267965d77d660'}

In [ ]:
# Parse articles

records = df.to_dict("records")

def extract_authors(text):
   ls_authors = []
   sep = ";"
   if ";" in text:
     sep = ";"
   else:
     sep = ","

   ls_authors.extend(text.split(sep))
   return ls_authors


articles = copy.deepcopy(records)
for r in articles:
    r["authors"] = extract_authors(r['authors'])

articles[20]


{'id': 'math/0001082',
 'authors': ['Alain Lascoux', ' Michel Lassalle (CNRS', 'Paris)'],
 'title': "Une identit\\'e remarquable en th\\'eorie des partitions",
 'journal': 'Math. Annalen, 318 (2000), 299-313',
 'update_date': '2007-05-23',
 'abstract': '  We prove an identity about partitions, previously conjectured in the study of\nshifted Jack polynomials (math.CO/9903020). The proof given is using\n$\\lambda$-ring techniques. It would be interesting to obtain a bijective proof.\n'}

In [ ]:

# Initialize a Neo4j Desktop session

from neo4j import GraphDatabase
pwd = "P5L4adkDdEPLh6jTz8uzVXOX96VMUmm-uo1HfYzaJ_E"
driver = GraphDatabase.driver('neo4j+s://30362ba7.databases.neo4j.io', auth=('neo4j', pwd))

In [ ]:
def read_query(query, params=None):
    with driver.session() as session:
        result = session.run(query, params)
        return pd.DataFrame([r.values() for r in result],
                            columns=result.keys())

In [ ]:

# Create Journal nodes
read_query("""
UNWIND $data as journal
MERGE (j:Journal {id:journal.id})
SET j.name = journal.name
RETURN count(j)
""",{"data":journals})

,count(j)
0,2550


In [ ]:
#Basic node retrieval
test_1 = "Fetch 5 journal names"
result_1 = gen_text(test_1)
query_1 =  """
MATCH (j:Journal)
RETURN j.name LIMIT 5
"""
print(result_1)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Provide answers in Cypher based on the following schema and example.

### The Schema

Nodes and their properties are the following:

{'properties': ['id', 'name'], 'labels': 'Journal'}
{'properties1': ['id', 'name'], 'labels': 'Author'}
{'properties': ['id', 'title', 'abstract', 'authors', 'date', 'journal'], 'labels': 'Article'}

The relationships are the following:

{'source': 'Journal', 'relationship': 'PUBLISHED', 'target': ['Article']}
{'source': 'Author', 'relationship': 'WROTE', 'target': ['Article']}

### Example

Question: Fetch 5 journal titles.
Cypher Query: MATCH (j:Journal) RETURN j.name LIMIT 5

<>

Fetch 5 journal names[/INST]

Here's an example answer:

Fetch 5 journal names:
```cypher
MATCH (j:Journal)
RETURN j.name
LIMIT 5
```
This query matches all nodes with the label `Journal` and returns the `name` property for each matching node. The `LIMIT` clause limits the number of results to 5.


In [ ]:
# Search through string property

test_2 = "Find 5 articles that contain algebra in the title and abstract."
query_2 = """
MATCH (a:Article)
WHERE a.abstract CONTAINS 'algebra' AND a.title CONTAINS 'algebra'
RETURN a.title as Title
LIMIT 5
"""
result_2 = gen_text(test_2)
print(result_2)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Provide answers in Cypher based on the following schema and example.

### The Schema

Nodes and their properties are the following:

{'properties': ['id', 'name'], 'labels': 'Journal'}
{'properties1': ['id', 'name'], 'labels': 'Author'}
{'properties': ['id', 'title', 'abstract', 'authors', 'date', 'journal'], 'labels': 'Article'}

The relationships are the following:

{'source': 'Journal', 'relationship': 'PUBLISHED', 'target': ['Article']}
{'source': 'Author', 'relationship': 'WROTE', 'target': ['Article']}

### Example

Question: Fetch 5 journal titles.
Cypher Query: MATCH (j:Journal) RETURN j.name LIMIT 5

<>

Find 5 articles that contain algebra in the title and abstract.[/INST]

Here is a Cypher query that finds 5 articles that contain the word "algebra" in the title and abstract:
```
MATCH (a:Article)
WHERE a.title CONTAINS "algebra" AND a.abstract CONTAINS "algebra"
RETURN a.title, a.abstract
LIMIT 5
```
This query uses the `CONTAINS` operator to check if the `title` and `abst

In [ ]:
# Search through string property

test_2 = "Find 5 articles that contain algebra in the title and abstract."

result_2 = gen_text(test_2)
print(result_2)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.




Provide answers in Cypher based on the following schema and example.

### The Schema

Nodes and their properties are the following:

{'properties': ['id', 'name'], 'labels': 'Journal'}
{'properties1': ['id', 'name'], 'labels': 'Author'}
{'properties': ['id', 'title', 'abstract', 'authors', 'date', 'journal'], 'labels': 'Article'}

The relationships are the following:

{'source': 'Journal', 'relationship': 'PUBLISHED', 'target': ['Article']}
{'source': 'Author', 'relationship': 'WROTE', 'target': ['Article']}

### Example

Question: Fetch 5 journal titles.
Cypher Query: MATCH (j:Journal) RETURN j.name LIMIT 5

<>

Find 5 articles that contain algebra in the title and abstract.[/INST]

Here is a Cypher query that should do what you're asking for:
```
MATCH (a:Article)
WHERE a.title CONTAINS "algebra" AND a.abstract CONTAINS "algebra"
RETURN a.title, a.abstract LIMIT 5
```
Explanation:

* `MATCH (a:Article)`: This matches any article node in the graph.
* `WHERE a.title CONTAINS "algebra

In [ ]:
query_2 = """
MATCH (a:Article)
WHERE a.abstract CONTAINS 'algebra' AND a.title CONTAINS 'algebra'
RETURN a.title as Title
LIMIT 5
"""

In [ ]:

def gen_(prompt, **kwargs):

    full_prompt = prompt_format.format(user=prompt)

    # The default max length is pretty small, increase the threshold
    if "max_new_tokens" not in kwargs:
        kwargs["max_new_tokens"] = 512

    output = pipeline(full_prompt,
                      do_sample=True,
                      top_k=10,
                      top_p=0.95,
                      **kwargs)

    return output

In [ ]:
test_4 = "Fetch all articles published in the journal Nature"

query_4 = """
MATCH (j:Journal {name: "Nature"})-[:PUBLISHED]->(a:Article)
RETURN a.title
"""

result_4 = gen_(test_4)
print(result_4)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
